In [ ]:
import spotipy 
import sys
from tensorflow import keras
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import time
import pandas as pd 
import numpy as np
import tkinter


cid ='74ec55c0be3c41d48da298607145034a' # Client ID; copy this from your app created on beta.developer.spotify.com
secret = '36242f76cc524f3b95ad51e22929f9a8' # Client Secret; copy this from your app
username = 'blendoruruci' # Your Spotify username

#for avaliable scopes see https://developer.spotify.com/web-api/using-scopes/
scope = 'user-library-read playlist-modify-public playlist-read-private'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
redirect_uri='https://developer.spotify.com/dashboard/applications/74ec55c0be3c41d48da298607145034a'
token = util.prompt_for_user_token(username, scope, cid, secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [ ]:
#Credentials to access the Spotify Music Data
manager = SpotifyClientCredentials(cid,secret)
sp = spotipy.Spotify(client_credentials_manager=manager)

#Credentials to access to  the Spotify User's Playlist, Favorite Songs, etc. 
token = util.prompt_for_user_token(username,scope,cid,secret,redirect_uri) 
spt = spotipy.Spotify(auth=token)

In [ ]:
model = keras.models.load_model("hitpredictionmodel.h5")

model

In [ ]:
def predict_hit(song_features):
        try:
            # Predict the song mood        
            pred = model.predict(song_features)
            
            pred = np.argmax(pred, axis = 1)

            # Map the moods to the val
            if pred == 0:
                hit = 'very low chance to become a hit at all'
            elif pred == 1:
                hit = 'low chance to become a hit'
            elif pred == 2:
                hit = 'medium chance to become a hit'
            elif pred == 3:
                hit = 'good chance to become a hit'
            elif pred == 4:
                hit = 'it is going to be a hit!'  

            return hit
        except:
            return "error"

def features(song_uri):

    track_features = []

    # get the song id from the uri 
    temp = song_uri.split("track/",1)[1]
    song_id = temp.split('?',1)[0]
    track, columns,track_data = get_songs_features(song_id)
    track_features.append(track)
    #song_features = pd.DataFrame(track_features,columns=columns)
    song_features = pd.DataFrame(track_features, columns=columns)
    song_features = song_features.astype('float32')
    #song_features = song_features.to_dict('records')[0]
    #song_features = song_features.values.tolist()
    

    hit = predict_hit(song_features)

    return hit,track_data

# END: Functions called from View.py
# ----------------------------------
   


# Functions called from spotify.py
# ----------------------------------

def get_songs_features(track_ids):

    meta = sp.track(track_ids)
    features = sp.audio_features(track_ids)
    track_data = prepate_metadata(meta)
    track_features, columns = prepare_featuredata(features)
    return track_features,columns,track_data

def prepate_metadata(meta):

    track = {
        'name': meta['name'],
        'artists': meta['artists'],
        'duration': meta['duration_ms'],
        'preview_url': meta['preview_url'],
        'id': meta['id']
    }

    return track

def prepare_featuredata(features):

        # features
    length = features[0]['duration_ms']
    danceability = features[0]['danceability']
    acousticness = features[0]['acousticness']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    valence = features[0]['valence']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']

    track_features = [length, danceability, acousticness,
            energy, instrumentalness, liveness, valence, loudness, speechiness, tempo]
    columns = ['length', 'danceability','acousticness','energy','instrumentalness',
                'liveness','valence','loudness','speechiness','tempo']

    return track_features,columns

def get_id_outof_link(url, entity):
    id = ""

    if entity == "playlist":
        temp = url.split("playlist/",1)[1]
        id = temp.split('?',1)[0]

    elif entity == "track":
        temp = url.split("track/",1)[1]
        id = temp.split('?',1)[0]

    elif entity == "album":
        temp = url.split("album/",1)[1]
        id = temp.split('?',1)[0]


    if id == "":
        return "Sorry, there is no ID we can find"
    return id

def convert_ms_to_min(ms):
    millis = ms
    seconds=(millis/1000)%60
    seconds = int(seconds)
    minutes=(millis/(1000*60))%60

    duration =  ("%d:%d" % (minutes, seconds))

    return duration

In [ ]:
## GUI Code
def ende():
    main.destroy()
def predictHit():
    eingabe = e.get()
    try:
        song_uri = str(eingabe)
        popularity,track_data = features(song_uri)
        lbe["text"] = popularity
    except:
        lbe["text"] = "Ein Fehler ist aufgetreten!"

main = tkinter.Tk()
e = tkinter.Entry(main)
e.pack()
lbz = tkinter.Label(main, text = "Geben sie die URL zu einem Album ein")
lbz.pack()
bzufall = tkinter.Button(main, text = "Hit predicten", command = predictHit)
bzufall.pack()
lbe = tkinter.Label(main, text = "Ergebnis:")
lbe.pack()
bende = tkinter.Button(main, text = "Ende", command = ende)
bende.pack()
main.mainloop()